# Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install dependencies

In [ ]:
!pip install -q diffusers transformers accelerate ftfy lpips einops torchmetrics Pillow tqdm xformers torch-fidelity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28

# Imports

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from torchvision.transforms import ToTensor, Resize
from torchmetrics.image.fid import FrechetInceptionDistance
import torchvision.transforms as transforms
from peft import PeftModel
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
from torch.nn import functional as F

/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


# Config

In [ ]:
# Device and base model
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BASE_MODEL = "runwayml/stable-diffusion-v1-5"

# Paths for LoRA weights
LORA_ROOT = "/content/drive/MyDrive/Colab Notebooks/sd_lora"
OUTPUT_DIR = f"{LORA_ROOT}/outputs/unet_only"
UNET_LORA_WEIGHTS = f"{OUTPUT_DIR}/checkpoint-step500/unet"
TEXT_ENCODER_LORA_WEIGHTS = f"{OUTPUT_DIR}/checkpoint-step500/text_encoder"

# Dataset paths for testing
TEST_IMAGES_DIR = f"{LORA_ROOT}/dataset/test/images"
TEST_CAPTIONS_DIR = f"{LORA_ROOT}/dataset/test/captions"

# Generation / testing parameters
RESOLUTION = 512
BATCH_SIZE = 2
NUM_INFERENCE_STEPS = 50          # DDIM / DPM solver steps for generation
SAVE_GENERATED_IMAGES = True

# Unified test output directory
TEST_OUTPUT_DIR = f"{LORA_ROOT}/testing/metrics_test/unet_only/checkpoint-step500"
GENERATED_DIR = f"{TEST_OUTPUT_DIR}/generated_images"
Path(GENERATED_DIR).mkdir(parents=True, exist_ok=True)

# Check if LoRA weights for text encoder exist (optional)
USE_TEXT_ENCODER_LORA = Path(TEXT_ENCODER_LORA_WEIGHTS).exists()

# Prepare test dataset

In [ ]:
class ImageCaptionDataset(Dataset):
    def __init__(self, images_dir, captions_dir, resolution=512):
        self.images_dir = Path(images_dir)
        self.captions_dir = Path(captions_dir)
        self.ids = sorted([p.stem for p in self.images_dir.glob("*.jpg")])
        self.resolution = resolution
        self.transform = transforms.Compose([
            transforms.Resize((resolution, resolution)),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)  # scale [-1,1]
        ])

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        id_ = self.ids[idx]
        img = Image.open(self.images_dir / f"{id_}.jpg").convert("RGB")
        img = self.transform(img)
        caption = (self.captions_dir / f"{id_}.txt").read_text(encoding="utf-8").strip()
        return {"image": img, "caption": caption, "id": id_}

# Load dataset
test_dataset = ImageCaptionDataset(TEST_IMAGES_DIR, TEST_CAPTIONS_DIR, RESOLUTION)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load Stable Diffusion + LoRA

In [ ]:
# Load base pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    safety_checker=None
).to(DEVICE)

# Load LoRA weights for UNet
pipe.unet = PeftModel.from_pretrained(pipe.unet, UNET_LORA_WEIGHTS).to(DEVICE)

# Optionally load LoRA weights for Text Encoder
if USE_TEXT_ENCODER_LORA:
    pipe.text_encoder = PeftModel.from_pretrained(pipe.text_encoder, TEXT_ENCODER_LORA_WEIGHTS).to(DEVICE)
    print("Loaded LoRA weights for Text Encoder.")

# Use DPM solver for faster inference
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Memory optimizations
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_attention_slicing()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Pl

ValueError: torch.cuda.is_available() should be True but is False. xformers' memory efficient attention is only available for GPU 

# Initialize metrics

In [ ]:
# CLIP setup
CLIP_MODEL_NAME = "openai/clip-vit-large-patch14"
clip_model = CLIPModel.from_pretrained(CLIP_MODEL_NAME).to(DEVICE)
clip_processor = CLIPProcessor.from_pretrained(CLIP_MODEL_NAME)

# FID metric
fid_metric = FrechetInceptionDistance(feature=2048).to(DEVICE)

# Transform helpers
to_tensor = ToTensor()
resize = transforms.Resize((RESOLUTION, RESOLUTION))

# Optional: create folder for generated images
if SAVE_GENERATED_IMAGES:
    Path(GENERATED_DIR).mkdir(parents=True, exist_ok=True)

# Generate images and compute metrics

In [ ]:


pipe.scheduler.set_timesteps(NUM_INFERENCE_STEPS)
pipe.enable_attention_slicing()  # reduce VRAM usage

clip_scores = []
clip_cos_sims = []

for batch in tqdm(test_loader, desc="Testing"):
    captions = batch["caption"]
    gt_images = batch["image"].to(DEVICE)  # [-1,1] scale

    # Generate images
    with torch.autocast("cuda"):
        gen_images = pipe(
            captions,
            height=RESOLUTION,
            width=RESOLUTION,
            num_inference_steps=NUM_INFERENCE_STEPS
        ).images

    # Convert generated images to tensor [-1,1]
    gen_tensors = torch.stack([to_tensor(resize(img)) * 2 - 1 for img in gen_images]).to(DEVICE)

    # ------------------ CLIPScore & cosine similarity ------------------
    # Preprocess for CLIP
    clip_inputs = clip_processor(
        text=captions,
        images=[Image.fromarray(((img.permute(1,2,0).cpu().numpy()+1)/2*255).astype("uint8")) for img in gen_tensors],
        return_tensors="pt",
        padding=True
    ).to(DEVICE)

    # Forward pass
    with torch.no_grad():
        clip_outputs = clip_model(**clip_inputs)
        image_embeds = F.normalize(clip_outputs.image_embeds, dim=-1)
        text_embeds = F.normalize(clip_outputs.text_embeds, dim=-1)

        # Cosine similarity
        cos_sim = (image_embeds * text_embeds).sum(dim=-1)
        clip_cos_sims.extend(cos_sim.cpu().tolist())

        # CLIPScore (scaled 0-100)
        clip_score = ((cos_sim + 1)/2 * 100)
        clip_scores.extend(clip_score.cpu().tolist())

    # ------------------ FID update ------------------
    gt_images_fid = ((gt_images + 1) * 127.5).clamp(0, 255).to(torch.uint8)
    gen_images_fid = ((gen_tensors + 1) * 127.5).clamp(0, 255).to(torch.uint8)
    fid_metric.update(gt_images_fid, real=True)
    fid_metric.update(gen_images_fid, real=False)

    # ------------------ Save generated images ------------------
    if SAVE_GENERATED_IMAGES:
        for img, id_ in zip(gen_images, batch["id"]):
            img.save(Path(GENERATED_DIR) / f"{id_}_gen.png")


# Compute final metrics

In [ ]:
final_fid = fid_metric.compute().item()
avg_clip_score = sum(clip_scores) / len(clip_scores)
avg_cos_sim = sum(clip_cos_sims) / len(clip_cos_sims)

# Prepare content
metrics_content = f"""
FID score: {final_fid:.4f}
Average CLIPScore (0-100): {avg_clip_score:.2f}
Average CLIP cosine similarity (-1 to 1): {avg_cos_sim:.4f}
"""

# Print metrics
print(metrics_content.strip())

# Save metrics as TXT
metrics_txt_path = Path(GENERATED_DIR).parent / "metrics.txt"
with open(metrics_txt_path, "w") as f:
    f.write(metrics_content.strip())

print(f"Metrics saved to {metrics_txt_path}")